In [ ]:
import sys
sys.path.insert(0, './algorithms/alpha_beta')
sys.path.insert(0, './algorithms/dqn')
sys.path.insert(0, './algorithms/stockfish')

In [ ]:
from algorithms.MyChessBoard import MyChessBoard
from algorithms.alpha_beta.Alpha_Beta import find_move as alpha_beta_find_move
from algorithms.dqn.dqn_bot import find_move as dqn_find_move
from algorithms.dqn.dqn_sf_bot import find_move as dqn_vs_sf_find_move
from algorithms.stockfish.stockfish_bot import find_move as stockfish_find_move

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,\
     Input,BatchNormalization, Dropout

In [ ]:
STATE_SHAPE = (65, )
NB_ACTIONS = 4096


In [ ]:
model_dqn = Sequential()
model_dqn.add(Input((1, ) + STATE_SHAPE))
model_dqn.add(Flatten())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(NB_ACTIONS))
model_dqn.add(Activation('linear'))
model_dqn.load_weights('chess_dqn_model.h5')


In [ ]:
model_dqn_sf = Sequential()
model_dqn_sf.add(Input((1, ) + STATE_SHAPE))
model_dqn_sf.add(Flatten())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(NB_ACTIONS))
model_dqn_sf.add(Activation('linear'))
model_dqn_sf.load_weights('chess_dqn_vs_sf_model.h5')

In [ ]:
board = MyChessBoard()

In [ ]:
board.convert_board_to_int()

array([[ -40.,  -20.,  -30.,  -50., -900.,  -30.,  -20.,  -40.],
       [ -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10.,   10.,   10.,   10.,   10.,   10.,   10.],
       [  40.,   20.,   30.,   50.,  900.,   30.,   20.,   40.]])

In [ ]:
state = board.get_state()
model_dqn.predict(state.reshape((1, 1) + STATE_SHAPE)).reshape(-1, )

array([-4125.982 , -3346.2021, -3559.5508, ..., -3943.0579, -3572.4902,
       -4092.5286], dtype=float32)

In [ ]:
dqn_find_move(board, model_dqn)

Move.from_uci('b1c3')

In [ ]:
name = ["Alpha_Beta", "DQN", "DQN_sf", "Stockfish"]
find = [alpha_beta_find_move, dqn_find_move, dqn_vs_sf_find_move, stockfish_find_move]

result = np.zeros((4, 4))
mean_step = np.zeros((4, 4))

In [12]:
num_battle_with_com1 = 50
com1 = 0

com2 = 1
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    step = 0
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


0 vs 1 : 0
0 vs 1 : 1
0 vs 1 : 2
0 vs 1 : 3
0 vs 1 : 4
0 vs 1 : 5
0 vs 1 : 6
0 vs 1 : 7
0 vs 1 : 8
0 vs 1 : 9
0 vs 1 : 10
0 vs 1 : 11
0 vs 1 : 12
0 vs 1 : 13
0 vs 1 : 14
0 vs 1 : 15
0 vs 1 : 16
0 vs 1 : 17
0 vs 1 : 18
0 vs 1 : 19
0 vs 1 : 20
0 vs 1 : 21
0 vs 1 : 22
0 vs 1 : 23
0 vs 1 : 24
0 vs 1 : 25
0 vs 1 : 26
0 vs 1 : 27
0 vs 1 : 28
0 vs 1 : 29
0 vs 1 : 30
0 vs 1 : 31
0 vs 1 : 32
0 vs 1 : 33
0 vs 1 : 34
0 vs 1 : 35
0 vs 1 : 36
0 vs 1 : 37
0 vs 1 : 38
0 vs 1 : 39
0 vs 1 : 40
0 vs 1 : 41
0 vs 1 : 42
0 vs 1 : 43
0 vs 1 : 44
0 vs 1 : 45
0 vs 1 : 46
0 vs 1 : 47
0 vs 1 : 48
0 vs 1 : 49
0 vs 2 : 0
0 vs 2 : 1
0 vs 2 : 2
0 vs 2 : 3
0 vs 2 : 4
0 vs 2 : 5
0 vs 2 : 6
0 vs 2 : 7
0 vs 2 : 8
0 vs 2 : 9
0 vs 2 : 10
0 vs 2 : 11
0 vs 2 : 12
0 vs 2 : 13
0 vs 2 : 14
0 vs 2 : 15
0 vs 2 : 16
0 vs 2 : 17
0 vs 2 : 18
0 vs 2 : 19
0 vs 2 : 20
0 vs 2 : 21
0 vs 2 : 22
0 vs 2 : 23
0 vs 2 : 24
0 vs 2 : 25
0 vs 2 : 26
0 vs 2 : 27
0 vs 2 : 28
0 vs 2 : 29
0 vs 2 : 30
0 vs 2 : 31
0 vs 2 : 32
0 vs 2 : 33
0 vs 2 : 34


In [14]:
com1 = 1
num_battle_with_com1 = 100

com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)

1 vs 2 : 0
fivefold repetition
1 vs 2 : 1
fivefold repetition
1 vs 2 : 2
fivefold repetition
1 vs 2 : 3
fivefold repetition
1 vs 2 : 4
fivefold repetition
1 vs 2 : 5
fivefold repetition
1 vs 2 : 6
fivefold repetition
1 vs 2 : 7
fivefold repetition
1 vs 2 : 8
fivefold repetition
1 vs 2 : 9
fivefold repetition
1 vs 2 : 10
fivefold repetition
1 vs 2 : 11
fivefold repetition
1 vs 2 : 12
fivefold repetition
1 vs 2 : 13
fivefold repetition
1 vs 2 : 14
fivefold repetition
1 vs 2 : 15
fivefold repetition
1 vs 2 : 16
fivefold repetition
1 vs 2 : 17
fivefold repetition
1 vs 2 : 18
fivefold repetition
1 vs 2 : 19
fivefold repetition
1 vs 2 : 20
fivefold repetition
1 vs 2 : 21
fivefold repetition
1 vs 2 : 22
fivefold repetition
1 vs 2 : 23
fivefold repetition
1 vs 2 : 24
fivefold repetition
1 vs 2 : 25
fivefold repetition
1 vs 2 : 26
fivefold repetition
1 vs 2 : 27
fivefold repetition
1 vs 2 : 28
fivefold repetition
1 vs 2 : 29
fivefold repetition
1 vs 2 : 30
fivefold repetition
1 vs 2 : 31
fivefo

In [15]:
com1 = 2
num_battle_with_com1 = 100

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


2 vs 3 : 0
2 vs 3 : 1
2 vs 3 : 2
2 vs 3 : 3
2 vs 3 : 4
2 vs 3 : 5
2 vs 3 : 6
2 vs 3 : 7
2 vs 3 : 8
2 vs 3 : 9
2 vs 3 : 10
2 vs 3 : 11
2 vs 3 : 12
2 vs 3 : 13
2 vs 3 : 14
2 vs 3 : 15
2 vs 3 : 16
2 vs 3 : 17
2 vs 3 : 18
2 vs 3 : 19
2 vs 3 : 20
2 vs 3 : 21
2 vs 3 : 22
2 vs 3 : 23
2 vs 3 : 24
2 vs 3 : 25
2 vs 3 : 26
2 vs 3 : 27
2 vs 3 : 28
2 vs 3 : 29
2 vs 3 : 30
2 vs 3 : 31
2 vs 3 : 32
2 vs 3 : 33
2 vs 3 : 34
2 vs 3 : 35
2 vs 3 : 36
2 vs 3 : 37
2 vs 3 : 38
2 vs 3 : 39
2 vs 3 : 40
2 vs 3 : 41
2 vs 3 : 42
2 vs 3 : 43
2 vs 3 : 44
2 vs 3 : 45
2 vs 3 : 46
2 vs 3 : 47
2 vs 3 : 48
2 vs 3 : 49
2 vs 3 : 50
2 vs 3 : 51
2 vs 3 : 52
2 vs 3 : 53
2 vs 3 : 54
2 vs 3 : 55
2 vs 3 : 56
2 vs 3 : 57
2 vs 3 : 58
2 vs 3 : 59
2 vs 3 : 60
2 vs 3 : 61
2 vs 3 : 62
2 vs 3 : 63
2 vs 3 : 64
2 vs 3 : 65
2 vs 3 : 66
2 vs 3 : 67
2 vs 3 : 68
2 vs 3 : 69
2 vs 3 : 70
2 vs 3 : 71
2 vs 3 : 72
2 vs 3 : 73
2 vs 3 : 74
2 vs 3 : 75
2 vs 3 : 76
2 vs 3 : 77
2 vs 3 : 78
2 vs 3 : 79
2 vs 3 : 80
2 vs 3 : 81
2 vs 3 : 82
2 vs 3 : 83
2 

In [16]:
result

array([[  0.,  50.,  50.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [ 50., 100., 100.,   0.]])

In [17]:
mean_steps

array([[   0.  ,   20.5 ,  395.  , 1401.34],
       [   0.  ,    0.  , 2823.25, 4121.2 ],
       [   0.  ,    0.  ,    0.  , 5456.11],
       [   0.  ,    0.  ,    0.  ,    0.  ]])